"""
Neuroimaging cartesian reconstruction
=====================================
Author: Chaithya G R
In this tutorial we will reconstruct an MRI image from the sparse kspace
measurements.
Import neuroimaging data
------------------------
We use the toy datasets available in pysap, more specifically a 2D brain slice
and the cartesian acquisition scheme.
"""


In [ ]:
# Package import
from modopt.math.metrics import ssim
from mri.numerics.fourier import FFT2
from mri.numerics.reconstruct import sparse_rec_condatvu
from mri.numerics.utils import generate_operators
from mri.numerics.utils import convert_mask_to_locations
from pysap.data import get_sample_data
import pysap

# Third party import
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading input data
image = get_sample_data('2d-mri')

# Obtain K-Space Cartesian Mask
mask = get_sample_data("cartesian-mri-mask")

# View Input
plt.figure()
plt.imshow(image, cmap='gray')
plt.figure()
plt.imshow(mask, cmap='gray')
plt.show()

Generate the kspace
-------------------

From the 2D brain slice and the acquisition mask, we retrospectively
undersample the k-space using a cartesian acquisition mask
We then reconstruct the zero order solution as a baseline

Get the locations of the kspace samples

In [ ]:
kspace_loc = convert_mask_to_locations(np.fft.fftshift(mask.data))
# Generate the subsampled kspace
fourier_op = FFT2(samples=kspace_loc, shape=image.shape)
kspace_data = fourier_op.op(image)

Zero order solution

In [ ]:
image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data),
                         metadata=image.metadata)
plt.imshow(np.abs(image_rec0), cmap='gray')
# Calculate SSIM
base_ssim = ssim(image_rec0, image)
print(base_ssim)

FISTA optimization
------------------

We now want to refine the zero order solution using a FISTA optimization.
The cost function is set to Proximity Cost + Gradient Cost

Generate operators

In [ ]:
gradient_op, linear_op, prox_op, cost_op = generate_operators(
    data=kspace_data,
    wavelet_name="sym8",
    samples=kspace_loc,
    nb_scales=4,
    mu=8 * 1e-7,
    non_cartesian=False,
    uniform_data_shape=None,
    gradient_space="analysis",
    padding_mode="periodization")

Start the FISTA reconstruction

In [ ]:
max_iter = 200
x_final, y_final, costs, metrics = sparse_rec_condatvu(
    gradient_op,
    linear_op,
    prox_op,
    cost_op,
    nb_of_reweights=0,
    max_nb_of_iter=max_iter,
    verbose=1)
image_rec = pysap.Image(data=np.abs(x_final))
plt.imshow(image_rec, cmap='gray')
# Calculate SSIM
recon_ssim = ssim(image_rec, image)
print('The Reconstruction SSIM is : ' + str(recon_ssim))
plt.show()